In [1]:
# import torch and libraries
import torch
import torch.nn as nn

In [2]:
# reuse functionality from previous notebooks
from utils import load_cifar

In [3]:
# if you don't have the cifar data yet
# !cifar2png cifar10 cifar10

In [5]:
images, labels = load_cifar('cifar10/train')

100%|██████████| 10/10 [00:11<00:00,  1.15s/it]
